In [ ]:
# this repository contais files to practise with julia

### 1 закрыть крест от исходного положения

In [7]:
using HorizonSideRobots

delta = [0,0] # X Y
# the following function also changes delta to return later
function moveDelta!(r::Robot, side::HorizonSide, delta::Vector)
    move!(r, side)
    x,y = delta
    if Int(side) == 1 y += 1
    elseif Int(side) == 2 x -= 1
    elseif Int(side) == 3 y -= 1
    else x += 1 end
    return [x,y]
end

function markLine!(r::Robot,side::HorizonSide)
    while isborder(r,side)==false 
        moveDelta!(r,side)
        putmarker!(r)
    end
    return [x,y]
end

function moveTimes!(r::Robot, side::HorizonSide, times::Integer, delta)
    for i in range(times):
        moveDelta!(r, side, delta)
    return [x,y]
end

function returnRobot(r::Robot, delta)
    x,y = delta
    if x > 0: moveTimes!(r, HorizonSide(2), x, delta) end
    if x < 0: moveTimes!(r, HorizonSide(4), x, delta) end
    if y > 0: moveTimes!(r, HorizonSide(3), y, delta) end
    if y < 0: moveTimes!(r, HorizonSide(1), y, delta) end
    return [x,y]
end
function cross!(r::Robot)
    for i in range(1, 4)
        side = HorizonSide(i)
        markLine!(r,side)
    end
    putmarker!(r)
end

inverse(side::HorizonSide) = HorizonSide(abs(Int(side)-2)) 
r = Robot("default.sit",animate=true)

# cross!(r)   

Robot(HorizonSideRobots.SituationDatas.SituationData((0x000000000000000b, 0x000000000000000c), 1.0, false, (6, 6), [381 283 … 216 106; 397 -90 … 491 -227; … ; 132 -202 … -228 218; -48 113 … 205 -30], Set{Tuple{Int64, Int64}}(), Set{HorizonSide}[Set() Set() … Set() Set(); Set() Set() … Set() Set(); … ; Set() Set() … Set() Set(); Set() Set() … Set() Set()], (Scene (780px, 715px):
  0 Plots
  1 Child Scene:
    └ Scene (780px, 715px), Axis (6 plots))), true)